#### Install all the Things

In [1]:
import pandas as pd

#### Read all the Data

Batting

In [2]:
batting_cols = ['playerID','yearID','teamID','H','BB','HBP','AB','SF']
batting = pd.read_csv('data/Batting.csv',usecols=batting_cols)

Salary

In [3]:
salary_cols = ['playerID','yearID','teamID','salary']
salary = pd.read_csv('data/Salaries.csv',usecols=salary_cols)

Player Names

In [4]:
player_cols = ['playerID','nameFirst','nameLast']
players = pd.read_csv('data/Master.csv',usecols=player_cols)

Positions

In [5]:
positions_cols = ['playerID', 'yearID', 'teamID', 'G_p', 'G_c',
                 'G_1b', 'G_2b', 'G_3b', 'G_ss', 'G_lf', 'G_cf',
                 'G_rf']
positions = pd.read_csv('data/Appearances.csv',usecols=positions_cols)

#### Calculate OBP in Batting DF, then remove unnecessary data

In [6]:
num = batting.H + batting.BB + batting.HBP
denom = batting.AB + batting.BB + batting.HBP + batting.SF
batting['OBP'] = num / denom

In [7]:
batting = batting.drop(['AB', 'H', 'BB', 'HBP', 'SF'],axis=1)
batting = batting.dropna()

#### Merge all the Things

In [8]:
batting_players = batting.merge(players,on='playerID')
batting_players.head()

,playerID,yearID,teamID,OBP,nameFirst,nameLast
0,aaronha01,1954,ML1,0.322068,Hank,Aaron
1,aaronha01,1955,ML1,0.366261,Hank,Aaron
2,aaronha01,1956,ML1,0.364885,Hank,Aaron
3,aaronha01,1957,ML1,0.377778,Hank,Aaron
4,aaronha01,1958,ML1,0.385542,Hank,Aaron


In [9]:
batting_players_pos = batting_players.merge(positions,on=['playerID', 'yearID', 'teamID'])
batting_players_pos.head()

,playerID,yearID,teamID,OBP,nameFirst,nameLast,G_p,G_c,G_1b,G_2b,G_3b,G_ss,G_lf,G_cf,G_rf
0,aaronha01,1954,ML1,0.322068,Hank,Aaron,0,0,0,0,0,0,105,0,11
1,aaronha01,1955,ML1,0.366261,Hank,Aaron,0,0,0,27,0,0,26,0,105
2,aaronha01,1956,ML1,0.364885,Hank,Aaron,0,0,0,0,0,0,0,0,152
3,aaronha01,1957,ML1,0.377778,Hank,Aaron,0,0,0,0,0,0,1,69,83
4,aaronha01,1958,ML1,0.385542,Hank,Aaron,0,0,0,0,0,0,0,39,120


In [10]:
master = batting_players_pos.merge(salary,on=['playerID', 'yearID', 'teamID'])
master = master.dropna()
master.head()

,playerID,yearID,teamID,OBP,nameFirst,nameLast,G_p,G_c,G_1b,G_2b,G_3b,G_ss,G_lf,G_cf,G_rf,salary
0,rosepe01,1985,CIN,0.394790,Pete,Rose,0,0,110,0,0,0,0,0,0,358858
1,rosepe01,1986,CIN,0.316176,Pete,Rose,0,0,61,0,0,0,0,0,0,1000000
2,staubru01,1985,NYN,0.400000,Rusty,Staub,0,0,0,0,0,0,0,0,1,300000
3,perezto01,1985,CIN,0.396135,Tony,Perez,0,0,50,0,0,0,0,0,0,225000
4,perezto01,1986,CIN,0.333333,Tony,Perez,0,0,55,0,0,0,0,0,0,275000


#### Create metric 'OBP/$' to determine players' values.

Since OBP is such a small number and salaries are so large, I multiply this metric by 100,000 just to make it easier to read. 

In [11]:
master['OBP/$'] = (master.OBP / master.salary) * 100000

#### Create Function to select best team from a given year.

In [12]:
def moneyball_team(df, year, min_games=50):
    roster = {'G_p':'Pitcher', 'G_c':'Catcher', 'G_1b':'First Base', 'G_2b':'Second Base',
              'G_3b':'Third Base', 'G_ss':'Shortstop', 'G_lf':'Left Field',
              'G_cf':'Center Field', 'G_rf':'Right Field'}
    team = []
    df_year = df[df.yearID == year].sort_values(by='OBP/$',ascending=False)
    for player, position in roster.items():
        current_player = df_year[df_year[player] >= min_games][:1]
        current_player['POS'] = position
        team.append(current_player)
    team = pd.concat(team)
    team = team.drop(['G_p', 'G_c', 'G_1b', 'G_2b',
                      'G_3b', 'G_ss', 'G_lf', 'G_cf', 'G_rf'], axis=1)
    team = team.set_index('playerID')
    team = team.sort_values(by='OBP/$',ascending=False)
    return team[['yearID', 'nameFirst','nameLast','POS','OBP', 'salary','OBP/$']]

#### Now, the moment of truth....   What is the best "moneyball" team we could put together in, say, 1990?

In [13]:
moneyball_MCMXC = moneyball_team(master, 1990)
moneyball_MCMXC

,yearID,nameFirst,nameLast,POS,OBP,salary,OBP/$
playerID,,,,,,,
deshide01,1990,Delino,DeShields,Second Base,0.374781,100000,0.374781
macksh01,1990,Shane,Mack,Right Field,0.391931,105000,0.373267
martied01,1990,Edgar,Martinez,Third Base,0.397188,108000,0.367767
diazed01,1990,Edgar,Diaz,Shortstop,0.337500,100000,0.337500
zeileto01,1990,Todd,Zeile,Catcher,0.333333,100000,0.333333
morriha02,1990,Hal,Morris,First Base,0.381381,114500,0.333084
milleke01,1990,Keith,Miller,Center Field,0.326923,100000,0.326923
dascedo01,1990,Doug,Dascenzo,Left Field,0.312030,100000,0.312030
castito02,1990,Tony,Castillo,Pitcher,0.250000,100000,0.250000


In [14]:
#money